# Graphing network packets

## Preparing data

The data source comes from a publicly available network forensics repository: http://www.netresec.com/?page=PcapFiles. The selected file is https://download.netresec.com/pcap/maccdc-2012/maccdc2012_00000.pcap.gz. We will select only TCP traffic.

```
tcpdump -qns 0 -r maccdc2012_00000.pcap | grep tcp > maccdc2012_00000.txt
```

For example, here is a snapshot of the resulting output:

```
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.24.100.1038 > 192.168.202.68.8080: tcp 0
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.27.100.37877 > 192.168.204.45.41936: tcp 0
09:30:07.780000 IP 192.168.24.100.1038 > 192.168.202.68.8080: tcp 0
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
09:30:07.780000 IP 192.168.202.68.8080 > 192.168.24.100.1038: tcp 1380
```

Given the directional nature of network traffic and the numerous ports per node, we will simplify the graph by treating traffic between nodes as undirected and ignorning the distinction between ports. The graph edges will have weights represented by the total number of bytes across both nodes in either direction.

```
python pcap_to_parquet.py maccdc2012_00000.txt
```

The resulting output will be two Parquet dataframes, `maccdc2012_nodes.parq` and `maccdc2012_edges.parq`.

## Loading data

In [ ]:
import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf

from colorcet import fire
from datashader.bundling import hammer_bundle
from datashader.layout import circular_layout

from dask.distributed import Client
from fastparquet import ParquetFile

client = Client()
width, height = 2000, 2000
x_range = (-0.01, 1.01)
y_range = (-0.01, 1.01)

In [ ]:
nodes_df = ParquetFile('maccdc2012_nodes.parq').to_pandas()
len(nodes_df)

In [ ]:
edges_df = ParquetFile('maccdc2012_edges.parq').to_pandas()
len(edges_df)

## Edge bundling

In [ ]:
positioned_df = circular_layout(nodes_df, edges_df)
positioned_df = positioned_df.set_index('id')

In [ ]:
bundled_df = hammer_bundle(positioned_df, edges_df)

In [ ]:
cvs = ds.Canvas(width, height, x_range, y_range)
img = tf.shade(cvs.points(bundled_df, 'x', 'y'), cmap=fire)
bundled_img = tf.set_background(img, color='black')

In [ ]:
bundled_img

## Showing nodes with active traffic

In [ ]:
active_edges_df = edges_df[edges_df['weight'] > 0]

In [ ]:
active_nodes = active_edges_df.source.append(active_edges_df.target, ignore_index=True).unique()
active_nodes_df = pd.DataFrame(active_nodes, columns=['id'])
active_nodes_df = active_nodes_df.set_index('id')
active_nodes_df = active_nodes_df.join(positioned_df)

In [ ]:
cvs = ds.Canvas(width, height, x_range, y_range)
agg = cvs.points(active_nodes_df, 'x', 'y')
nodes_img = tf.spread(tf.shade(agg, cmap='red'), px=5)

In [ ]:
bundled_img + nodes_img